In [10]:
import sys
!{sys.executable} -m pip install --upgrade pymorphy2

from pymorphy2 import MorphAnalyzer

parser = MorphAnalyzer()

In [11]:
import requests
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

# Загрузка файла
url = "https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt"
response = requests.get(url)
rus_stops = response.text.splitlines()

punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'
filter_token = rus_stops + list(punctuation)

parser = MorphAnalyzer()

def preprocess_gemsim(input_text):
    text = input_text.lower()
    tokenized_text = word_tokenize(text)
    clean_text = [word for word in tokenized_text if word not in filter_token]
    lemmatized_text = [parser.parse(word)[0].normal_form for word in clean_text if word]  # Проверка на пустые строки
    return lemmatized_text


<>:10: SyntaxWarning: invalid escape sequence '\]'
<>:10: SyntaxWarning: invalid escape sequence '\]'
C:\Users\Сократ\AppData\Local\Temp\ipykernel_18052\1493062947.py:10: SyntaxWarning: invalid escape sequence '\]'
  punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'


In [12]:
import pandas as pd

# Открытие файла с явным указанием кодировки
with open("anek_utf8.txt", "r", encoding="utf-8") as file1:
    lines = []
    for line in file1:
        if line.strip():  # Проверка на пустую строку
            lines.append(line.strip())  # Удаление пробелов и символов новой строки

line_new = []
for i in range(len(lines) - 1):  # Используем 'i' вместо 'n'
    if lines[i][:15] == '<|startoftext|>':
        if lines[i + 1][:15] != '<|startoftext|>':
            line_new.append(lines[i][15:] + ' ' + lines[i + 1][15:])
        else:
            line_new.append(lines[i][15:])

# Создание DataFrame
ds_line = pd.DataFrame(line_new, columns=['text_base'])


In [13]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')#для табов

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [14]:
import nltk
nltk.download('punkt')

# ds_line["text_processed_new"] = ds_line["text_base"].apply(lambda text: preprocess_sklearn(text))
ds_line["text_processed"] = ds_line["text_base"].apply(lambda text: preprocess_gemsim(text))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
# Build the bigram  models
texts=(ds_line["text_processed"])
from gensim import corpora, models

bigram = models.Phrases(texts, min_count=3, threshold=5)
bigram_mod = models.phrases.Phraser(bigram)
texts_bi = make_bigrams(texts)
dictionary_bi = corpora.Dictionary(texts_bi)                 # составляем словарь из терминов с учетом биграмм слов предлжений
print('Размер словаря до фильтрации: {}'.format(len(dictionary_bi)))
dictionary_bi.filter_extremes(no_below=3, no_above=0.4, keep_n=3*10**6)
print('Размер словаря после фильтрации: {}'.format(len(dictionary_bi)))
corpus_bi = [dictionary_bi.doc2bow(text) for text in texts_bi]  # составляем корпус документов по словарю полученному с использованием биграмм

Размер словаря до фильтрации: 83955
Размер словаря после фильтрации: 37948


In [ ]:
number_topics=20

In [ ]:
ldamodel_bi=lda_model_gensim(corpus_bi,dictionary_bi,number_topics)


In [ ]:
# topics_gensim_base = ldamodel_base.show_topics(num_topics=number_topics, num_words=100,formatted=False)
topics_gensim_bi = ldamodel_bi.show_topics(num_topics=number_topics, num_words=100,formatted=False)
with open('topics_base_cloud', 'wb') as f:
    pickle.dump(topics_gensim_base, f)
with open('topics_bi_cloud', 'wb') as f:
    pickle.dump(topics_gensim_bi, f)

topics_base=pd.DataFrame(topics_gensim_base)
topics_base.columns=['topic num','topic_word']
topics_bi=pd.DataFrame(topics_gensim_bi)
topics_bi.columns=['topic num','topic_word']

NameError: name 'ldamodel_base' is not defined